<a href="https://colab.research.google.com/github/MikolajKita/Neural-Networks/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip p4.zip

Archive:  p4.zip
  inflating: test_no_target.pkl      
  inflating: train.pkl               
  inflating: treШЖ_zadania.txt     


In [32]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [77]:
import pickle


with open('train.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [78]:
device = torch.device('cuda') # 
#device = torch.device('cpu')
device

device(type='cuda')

In [79]:
torch.manual_seed(42)
if torch.cuda.is_available(): 
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    
# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

In [80]:
torch.set_default_dtype(torch.float64)

In [81]:
type(dataset)

list

In [82]:
len(dataset)

2939

In [83]:
dataset[642][1]



0

In [86]:
#{0: 'bach', 1: 'beethoven', 2: 'debussy', 3: 'scarlatti', 4: 'victoria'}.

class_bach_0 = 0
class_beethoven_1 = 0
class_debussy_2 = 0
class_scarlatti_3 = 0
class_victoria_4 = 0


In [87]:
for i in range (0,len(dataset)):
  if(dataset[i][1] == 0):
    class_bach_0 += 1
  if(dataset[i][1] == 1):
    class_beethoven_1 += 1
  if(dataset[i][1] == 2):
    class_debussy_2 += 1
  if(dataset[i][1] == 3):
    class_scarlatti_3 += 1
  if(dataset[i][1] == 4):
    class_victoria_4 += 1

In [88]:
class_bach_0


1630

In [89]:
class_beethoven_1


478

In [90]:
class_debussy_2


154

In [91]:
class_scarlatti_3


441

In [92]:
class_victoria_4

236

In [93]:
from torch.utils.data import Dataset, DataLoader
class VariableLenDataset(Dataset):
    def __init__(self, in_data):
        self.data = [(x, y) for x, y in in_data]    
        
    def __len__(self):
        return len(self.data)

    def max(self):
        max = 0
        for data, target in self.data:
            if(len(data) > max):
              max = len(data)
        return max
          

    def __getitem__(self, idx):
        seq, target = self.data[idx]
        return seq, target

In [94]:
train_part = int(0.8 * len(dataset))
train_size = int(0.8 * train_part)
validation_size = train_part - train_size
test_size = len(dataset) - train_part
train_set, validation_set, test_set = torch.utils.data.random_split(dataset, [train_size, validation_size, test_size])

In [54]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
def collate_fn(batch, pad_value=0):
    """
       data: is a list of tuples with (example, label, length)
             where 'example' is a tensor of arbitrary shape
             and label/length are scalars
    """
    #print(batch[1])
    #inputs = [torch.tensor(d['tokenized_input']) for d in data] #(3)
   # labels = [d['label'] for d in data]
    seq, target = zip(*batch)
    inputs = [torch.tensor(x) for x in seq] #(3)
    labels = torch.tensor(target)
    seq_pad = pad_sequence(inputs, batch_first=True)
    return seq_pad, labels

In [111]:
train_loader = DataLoader(train_set, batch_size=50, shuffle=True, drop_last = True, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=50, shuffle=False, drop_last = True, collate_fn=collate_fn)
validation_loader = DataLoader(validation_set, batch_size=50, shuffle=False, drop_last = True, collate_fn=collate_fn)

In [97]:
len(train_set)

1880

In [101]:
len(validation_set)

471

In [100]:
len(test_set)

588

In [103]:
train_set[0]

(array([ 73.,  73.,  80.,  80.,   2.,  80.,  80.,  36.,  92.,  93.,  47.,
         88.,  88.,  92., 153.,  47., 180.,  73.,   2., 114.,  88.,   0.,
        100., 153.,  80., 168., 158., 175.,  13.,  92.,  92.,   5.,  92.,
         47.,  73.,  88.,  88.,  92., 153.,  47., 180.,  47., 172.,  73.,
         78.,  60., 185., 185., 185.,   8.,  12., 117., 159., 185., 185.,
         98.,   0.,   8.,   8.,  73., 117.,   0.,  36.,  78.,  36.,  36.,
         47.,  12.,  12.,  47.,  12., 185.,  12.,  47.,  30.,  44.,  78.,
         78.,  44.,  44.,  30.,  30.,  44.,  44.,  44.,  60.,  60.,  60.,
         44.,  30.,  25.,  20.,  79.,  79., 185., 185.,  12.,  73.,  73.,
         92.,   5.,   5.,  93.,  61., 140., 109.,  79.,  79.,  79., 112.,
        146., 120.,  32.,  37.,  37.,   9.,   9., 148., 114., 140.,  60.,
        158., 140., 172.,  79.,  79., 159.,  12., 159., 159.,  80., 159.,
         28., 156., 111., 137., 137.,  45., 159.,  92.,  33.,  41.,  12.,
        159., 125.,   8.,   0., 158., 

In [104]:
train_features, train_labels = next(iter(train_loader))

In [105]:
train_features[0].size()

torch.Size([2645])

In [106]:
train_features[1].size()

torch.Size([2645])

In [109]:
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, out_size)
        
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, state
    
    def forward(self, x, hidden):
        x = torch.transpose(x, 0, 1)
        all_outputs, hidden = self.lstm(x, hidden)
        out = all_outputs[-1] # We are interested only in the last output
        x = self.fc(out)
        return x, hidden
    
model = LSTMRegressor(input_size=1, hidden_size = 100, num_layers = 6, out_size = 5).to(device)
model

LSTMRegressor(
  (lstm): LSTM(1, 100, num_layers=6)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)

In [110]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fun = nn.CrossEntropyLoss()

training_loss = 0
validation_loss = 0
num_epochs = 20
training_loss = 0
validation_loss = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    for i, data in enumerate(train_loader, 0):
      #print(i, data)
      #print(i, data)

      inputs, labels = data
      inputs = inputs.to(device).unsqueeze(2)
      #inputs = [seq.to(device) for seq in inputs]
      labels = labels.to(device)
      #print(inputs)
      hidden, state = model.init_hidden(inputs.size(0))
      hidden, state = hidden.to(device), state.to(device)
      preds, last_hidden = model(inputs, (hidden,state))
      optimizer.zero_grad() 
      loss = loss_fun(preds, labels)
      loss.backward()
      optimizer.step()
      training_loss += loss.item()

    model.eval()
    with torch.no_grad():
      for i, data in enumerate(validation_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device).unsqueeze(2)
        labels = labels.to(device)
        #hidden, state = hidden.to(device), state.to(device)
        preds, last_hidden = model(inputs, (hidden,state))
        loss = loss_fun(preds, labels)
      
        validation_loss += loss.item()

    if epoch % 5 == 0:
      print(f"Epoch: {epoch}, training loss: {training_loss/1:.3}, validation loss: {validation_loss/1:.3}")
      training_loss = 0
      validation_loss = 0

Epoch: 0, training loss: 49.7, validation loss: 11.5
Epoch: 5, training loss: 2.37e+02, validation loss: 57.4
Epoch: 10, training loss: 2.33e+02, validation loss: 57.2
Epoch: 15, training loss: 2.34e+02, validation loss: 57.3


In [113]:
correct_pred = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
total_pred = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
correct_pred.items()

dict_items([(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)])

In [117]:
acc_sum = 0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device).unsqueeze(2)
        labels = labels.to(device)
        #hidden, state = hidden.to(device), state.to(device)
        preds, last_hidden = model(inputs, (hidden,state))
        _, predictions = torch.max(preds, 1)
        for label, prediction in zip(labels, predictions):
          if label == prediction:
              correct_pred[label.item()] += 1
          total_pred[label.item()] += 1



In [118]:
correct_pred.items()

dict_items([(0, 306), (1, 0), (2, 0), (3, 0), (4, 0)])

In [119]:
total_pred.items()

dict_items([(0, 306), (1, 87), (2, 28), (3, 84), (4, 45)])

In [145]:
acc_sum = 0
for classname, correct_count in correct_pred.items():
    print("Class: {}, correct predictions: {}, total_predictions {}".format(classname, correct_count, total_pred[classname]))
    accuracy = 100 * float(correct_count) / total_pred[classname]
    acc_sum += accuracy
    print("Accuracy for class {} is: {:.1f} %".format(classname, 
                                                   accuracy))

Class: 0, correct predictions: 306, total_predictions 306
Accuracy for class 0 is: 100.0 %
Class: 1, correct predictions: 0, total_predictions 87
Accuracy for class 1 is: 0.0 %
Class: 2, correct predictions: 0, total_predictions 28
Accuracy for class 2 is: 0.0 %
Class: 3, correct predictions: 0, total_predictions 84
Accuracy for class 3 is: 0.0 %
Class: 4, correct predictions: 0, total_predictions 45
Accuracy for class 4 is: 0.0 %


In [141]:
acc_sum

100.0

In [143]:
acc_average_all_classes = acc_sum / len(correct_pred)
acc_average_all_classes

20.0